In [178]:
%matplotlib inline 

import matplotlib.pyplot as plt 
import pandas 
from numpy import sqrt, array, nansum, linspace
from itertools import product 
from numpy import linalg as LA

In [179]:
cols = [ 'record', 'atom_number', 'atom_name', 'residue', 'residue_number', 'x', 'y', 'z', 'occ', 'b', 'element' ] 
bglb = pandas.read_csv( 'bglb_atoms.pdb', sep='\s+', header=None, names=cols )
print bglb.sample( 5 ) 

      record atom_number atom_name residue  residue_number       x       y  \
ATOM    1996          HA       ASP       A             128  57.940  29.572   
ATOM    6613        2HG2       ILE       A             420  83.133  22.560   
ATOM    3821          HE       ARG       A             246  48.599  41.084   
ATOM    6551          CB       ILE       A             417  81.243  30.040   
ATOM     787         NE2       HIS       A              56  81.839  35.362   

           z  occ    b element  
ATOM  62.561  1.0  0.0       H  
ATOM  43.991  1.0  0.0       H  
ATOM  32.204  1.0  0.0       H  
ATOM  33.433  1.0  0.0       C  
ATOM  47.596  1.0  0.0       N  


In [180]:
print len( bglb ), 'atoms to consider' 

7035 atoms to consider


In [181]:
# adding data about residues within 12 A of ligand in model 

! awk '{ print $6 }' bglb_active_site.pdb | sort -u > active_site_residues.txt 

In [182]:
with open( 'active_site_residues.txt' ) as fn:
    print ', '.join( fn.read().split() )

114, 116, 117, 118, 119, 120, 121, 122, 13, 14, 15, 16, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 17, 170, 171, 172, 175, 176, 177, 178, 179, 18, 19, 195, 20, 21, 217, 218, 219, 220, 221, 222, 223, 224, 240, 243, 244, 245, 289, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 313, 315, 320, 321, 322, 323, 324, 325, 326, 327, 33, 332, 335, 34, 350, 351, 352, 353, 354, 355, 356, 357, 358, 37, 372, 375, 376, 379, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 420, 422, 43, 430, 44, 45, 48, 50, 53, 54, 75, 76, 77, 78, 79, 80


In [183]:
active_site_residues = [ 114, 116, 117, 118, 119, 120, 121, 122, 13, 14, 15, 16, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 17, 170, 171, 172, 175, 176, 177, 178, 179, 18, 19, 195, 20, 21, 217, 218, 219, 220, 221, 222, 223, 224, 240, 243, 244, 245, 289, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 313, 315, 320, 321, 322, 323, 324, 325, 326, 327, 33, 332, 335, 34, 350, 351, 352, 353, 354, 355, 356, 357, 358, 37, 372, 375, 376, 379, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 420, 422, 43, 430, 44, 45, 48, 50, 53, 54, 75, 76, 77, 78, 79, 80 ] 

In [184]:
bglb = bglb[ ( bglb.residue_number.isin( active_site_residues ) ) ]
print len( bglb ), 'atoms to consider' 

2051 atoms to consider


In [185]:
charges = { 'C': -0.3, 'N': 1, 'O': -1, 'H': 0.1 }
bglb[ 'charge' ] = bglb.element.map( charges ) 

In [186]:
my_my = []
for i, s in bglb.iterrows():
    my_my.append( array( [ s.x, s.y, s.z ] ) ) 
bglb[ 'xyz' ] = my_my 

In [187]:
print bglb.sample( 5 ) 

      record atom_number atom_name residue  residue_number       x       y  \
ATOM    1889         1HB       LEU       A             122  62.682  34.115   
ATOM    6392          HA       TRP       A             407  68.225  24.613   
ATOM    3424          HA       ASN       A             220  62.755  35.584   
ATOM    5086         1HG       GLU       A             326  63.943  24.084   
ATOM     298           O       ILE       A              20  79.520  28.943   

           z  occ    b element  charge                       xyz  
ATOM  53.600  1.0  0.0       H     0.1    [62.682, 34.115, 53.6]  
ATOM  46.328  1.0  0.0       H     0.1  [68.225, 24.613, 46.328]  
ATOM  35.796  1.0  0.0       H     0.1  [62.755, 35.584, 35.796]  
ATOM  23.556  1.0  0.0       H     0.1  [63.943, 24.084, 23.556]  
ATOM  48.905  1.0  0.0       O    -1.0   [79.52, 28.943, 48.905]  


In [188]:
k = 1 # Coulomb's constant
# from numpy import pi 
# E = what?
# k = 1 / 4 * pi * E 

In [189]:
# charge at any point is the sum of all charges V
# V = kQ/r where k is Coulomb's constant, Q is charge of other particle and r is distance 

my_field = []
n_samples = 32 

my_x_range = linspace( bglb.x.min(), bglb.x.max(), n_samples )
my_y_range = linspace( bglb.y.min(), bglb.y.max(), n_samples )
my_z_range = linspace( bglb.z.min(), bglb.z.max(), n_samples )
pp = [ array( i ) for i in product( my_x_range, my_y_range, my_z_range ) ] 

print len( pp ), 'point samples considered' 

for p in pp:
    # then the charge experienced from that point is 
    # the sum of individual charges 
    individual_charges = []
    print '.',

    for index, series in bglb.iterrows():
        r = LA.norm( p - series.xyz )         
        Q = series.charge 
        my_charge = ( k * Q ) / r 
        individual_charges.append( my_charge ) 

    my_total = array( individual_charges )
    my_sum = nansum( my_total )
    
    my_field.append( ( p, my_sum ) ) 

216 point samples considered
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [190]:
# write a hokey PDB for visualization 
atoms = [ 'ATOM      1  X   XXX X  69      {0:2.3f}  {1:2.3f}  {2:2.3f}  1.00  {3:1.2f}           N \n'.format( xyz[0], xyz[1], xyz[2], c ) for xyz, c in my_field ] 

with open( 'the_grid.pdb', 'w' ) as fn:
    fn.write( ''.join( atoms ) )

In [191]:
# run and then the_grid > C > spectrum > b-factors
# to see the calculated potential field 
!pymol -d 'cd /Users/alex/Documents/bagel-benchmark/point_charges/; load bglb.pdb; load the_grid.pdb; color gray50, bglb; show spheres, organic;'

 PyMOL(TM) Molecular Graphics System, Version 1.7.6.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

 Detected OpenGL version 2.0 or greater. Shaders available.
 Detected GLSL version 1.20.
 OpenGL graphics engine:
  GL_VENDOR:   Intel Inc.
  GL_RENDERER: Intel HD Graphics 5000 OpenGL Engine
  GL_VERSION:  2.1 INTEL-10.14.73
 Detected 4 CPU cores.  Enab

ValueError: I/O operation on closed file